In [54]:
import os
from typing import List
from crewai import Agent, Crew, Process, Task, LLM
from crewai.project import CrewBase, agent, crew, task
from crewai_tools import WebsiteSearchTool, ScrapeWebsiteTool, TXTSearchTool, DirectorySearchTool
from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings

In [55]:
load_dotenv()

openai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
openai_api_version = os.getenv("OPENAI_API_VERSION")

llm_tool_user = AzureChatOpenAI(       
    azure_deployment="gpt-4o",       
    model="gpt-4o", 
    max_tokens=None,
    timeout=None,
    api_version=os.getenv("OPENAI_API_VERSION"),  
)

llm_tool_user_agent = Agent(
    role='Directory_Researcher',
    goal='Search Directory for files',
    backstory="Ein Assistent der bei der Sucher von Dateien hilft.",    
    llm=llm_tool_user
)

llm_chat_manager = AzureChatOpenAI(   
    azure_deployment="gpt-4o",       
    model="gpt-4o", 
    max_tokens=None,
    timeout=None,
    api_version=os.getenv("OPENAI_API_VERSION"),  
)

llm_chat_manager_agent = Agent(
    role='Chat_Researcher',
    goal='Findet entsprechend der Anfrage den optimalen Agent für die Beantwortung',
    backstory="Ein erfahren Leader, der das Team optimal einsetzt und rückmeldungen kritisch hinterfragt.",    
    llm=llm_tool_user
)

llm_rag_dir_search = AzureChatOpenAI(   
    azure_deployment="gpt-4o",       
    model="gpt-4o",  
    max_tokens=None,
    timeout=None,
    api_version=os.getenv("OPENAI_API_VERSION"),  
)

# Pass the text chunks to the Embedding Model from Azure OpenAI API to generate embeddings.
embedding_rag_dir_search = AzureOpenAIEmbeddings(    
    azure_deployment="text-embedding-3-large",
    model="text-embedding-3-large",        
    api_version=os.getenv("OPENAI_API_VERSION"),
    )  

# llm = AzureOpenAI(    
#     model="gpt-4o",
#     temperature=0.1,
#     max_tokens=150,
#     top_p=0.9,
#     frequency_penalty=0.0,
#     presence_penalty=0.0,
#     stop=["END"],
#     base_url= os.getenv("AZURE_OPENAI_ENDPOINT"),
#     deploymansss = "",
#     api_key=os.getenv("OPENAI_API_KEY"),
#     api_version=os.getenv("OPENAI_API_VERSION"),
#     )


# chat_manager = LLM(
#     model="gpt-4o",
#     temperature=0.3,
#     max_tokens=150,
#     top_p=0.9,
#     frequency_penalty=0.0,
#     presence_penalty=0.0,
#     stop=["END"],
#     base_url= os.getenv("AZURE_OPENAI_ENDPOINT"),
#     api_key=os.getenv("OPENAI_API_KEY"),
#     api_version=os.getenv("OPENAI_API_VERSION"),
# )


In [58]:
tool = DirectorySearchTool("C:\\Repos\\DevOps\\HC-Release-PV\\Product_MED\\Product_MED_AL_DACH\\_Work\\Script",                            
                           llm=llm_rag_dir_search, 
                           embedder=embedding_rag_dir_search)

Inserting batches in chromadb:   0%|          | 0/2 [00:00<?, ?it/s]


In [59]:
task = Task(
        description="Wieviele Powershell Scripte sind enthalten?",
        expected_output="Die anzahl der verwendet powershell scripte in der korrekten menge",       
        agent=llm_tool_user_agent,
    )

In [61]:
# Establishing the crew with a hierarchical process and additional configurations
project_crew = Crew(
    tasks=[task], 
    agents=[llm_tool_user_agent],
    #manager_llm=llm_chat_manager,  # Mandatory if manager_agent is not set
    process=Process.hierarchical,  # Specifies the hierarchical management approach
    respect_context_window=True,  # Enable respect of the context window for tasks
    memory=True,  # Enable memory usage for enhanced task execution
    manager_agent=llm_chat_manager_agent,  # Optional: explicitly set a specific agent as manager instead of the manager_llm
    planning=False,  # Enable planning feature for pre-execution strategy
    verbose=True,
)

result = project_crew.kickoff()
print(result)

2024-10-07 17:26:08,044 - 10236 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: c247debc********************66f7. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}